In [1]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [2]:
# loading environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
code_gemma = "google/codegemma-7b-it"
CODE_GEMMA_URL = "https://c5hggiyqachmgnqg.us-east-1.aws.endpoints.huggingface.cloud"

In [5]:
system_message = "Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M2. "
system_message += "Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
system_message += "La respuesta C++ debe producir una salida idéntica en el menor tiempo posible."

In [6]:
def user_prompt_for(python):
    user_prompt = "Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible."
    user_prompt += "Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios."
    user_prompt += "Manten la implementación de la generación de números aleatorios idénticos para que los resultados de la coincidencia sean exactos."
    user_prompt += "Responde solo con código C++; no añadas nada más que código; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
    user_prompt += "Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes de C++ necesarios, como iomanip.\n\n"    
    user_prompt += python
    return user_prompt

In [7]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [8]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [ ]:
# Stream

def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')
        
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')
            
def stream_code_gemma(python):
    tokenizer = AutoTokenizer.from_pretrained(code_gemma)
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_GEMMA_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=700)
    result = ""
    for r in stream:
        result += r.token.text
        yield result  

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="CodeGemma":
        result = stream_code_gemma(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far  

In [ ]:
# Declare execution code

def write_output(cpp):
  code = cpp.replace("```cpp","").replace("```","")
  with open("optimized.cpp", "w") as f:
      f.write(code)

def execute_python(code):
  try:
      output = io.StringIO()
      sys.stdout = output
      exec(code)
  finally:
      sys.stdout = sys.__stdout__
  return output.getvalue()

def execute_cpp(code):
  write_output(code)
  try:
      compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
      compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
      run_cmd = ["./optimized"]
      run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
      return run_result.stdout
  except subprocess.CalledProcessError as e:
      return f"An error occurred:\n{e.stderr}"

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        code = gr.Dropdown(["Pi", "Python hard"], label="Selecciona el el código", value="Pi")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "CodeGemma"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    code.change(update_code, inputs=[code], outputs=[python])
    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

In [ ]:
ui.launch(inbrowser=True)